# Weekly activity

In [3]:
import cv2 as cv
import numpy as np
from utils import display_image, display_images

if not cv.useOptimized():
    cv.setUseOptimized(True)

cv.useOptimized()

True

1. Rotate image by 45 degrees without cropping the sides of the image. (Hint: There are 2 strategies to tackle these problems). Use _"lena.jfif"_ as the input image.
    - Use external libraries `imutils`.  
    - Modify the transformation matrix.

In [5]:
pip install imutils

Note: you may need to restart the kernel to use updated packages.


In [85]:
import imutils

img = cv.imread("images/lena.jfif")

angle = 45

dst = imutils.rotate_bound(img, angle)

display_images([img, dst], ("original", "rotate"))

In [11]:
img = cv.imread("images/lena.jfif")

h, w = img.shape[:2]
center = (w//2, h//2)
angle = 45
scale = 1
M = cv.getRotationMatrix2D(center, angle, scale)

abs_cos = abs(M[0, 0])
abs_sin = abs(M[0, 1])
new_w = int(h*abs_sin + w*abs_cos)
new_h = int(h*abs_cos + w*abs_sin)

M[0, 2] += (new_w/2) - center[0]
M[1, 2] += (new_h/2) - center[1]

dst = cv.warpAffine(img, M, (new_w, new_h))
display_images([img, dst], ("original", "rotate"))

2. Use the images with titles: _"flower.jfif"_ and _"native-bee.png"_. I want to put flower above an image. If I add two images, it will change color. If I blend it, I get a transparent effect. But I want it to be opaque. If it was a rectangular region, we could use the ROI as we did in the previous section. But flower is not a rectangular region. This is where bitwise operations, like AND, OR, NOT and XOR really come in handy. The associated functions are `cv.bitwise_and()`, `cv.bitwise_or()` and `cv.bitwise_not()`. You need to use `cv.threshold` function to segment the flower. Please refer to [online documentation](https://docs.opencv.org/4.x/d0/d86/tutorial_py_image_arithmetics.html) for more info. The result should resemble the following:  
![bee and flowers](img_embed/activity3.PNG "bee_flower")

In [21]:
bee = cv.imread("images/native-bee.png")
flower = cv.imread("images/flower.jfif")

rows, cols, cahnnels = flower.shape   # get dimension of flower img
roi = bee[0:rows, 0:cols]   # define roi in bee (the place flower will be placed)

flower_gray = cv.cvtColor(flower, cv.COLOR_BGR2GRAY)    # convert flower image to grayscale
ret, mask = cv.threshold(flower_gray, 10, 255, cv.THRESH_BINARY)     # create binary mask
mask_inv = cv.bitwise_not(mask)    # create inverse mask

bee_bg = cv.bitwise_and(roi, roi, mask = mask_inv)     # black out area of flower in bee
flower_fg = cv.bitwise_and(flower, flower, mask = mask)    # take region of flower

# add flower to roi & modify main image
dst = cv.add(bee_bg, flower_fg)
bee[0:rows, 0:cols] = dst

display_image("flower_bee", bee)

3. Write a function that randomly crop the central region of an image. The method signature should be as shown in the following:
```
random_center_crop(image, min_crop_ratio, max_crop_ratio)
```

In [87]:
def random_center_crop(image, min_crop_ratio, max_crop_ratio):
    """
    Parameter
    ---
    image: image to be cropped
    min_crop_ratio: minimum crop ratio
    max_crop_ratio: maximum crop ratio

    Returns
    ---
    img_crop: cropped image
    """

    if not ( 0 <= min_crop_ratio <= max_crop_ratio <= 1):
        raise ValueError("Crop ratios must between 0 and 1")

    # get dimensions
    h, w = img.shape[:2]

    #select random crop ratio within range
    crop_ratio = np.random.uniform(min_crop_ratio, max_crop_ratio)
    
    # get new dimensions
    new_w = int(w * crop_ratio)
    new_h = int(h * crop_ratio)

    # get top left coordinate of cropped region
    top = (h - new_h) // 2
    left = (w - new_w) // 2

    # crop central region
    img_crop = img[top:top + new_h, left:left + new_w]

    return img_crop

In [43]:
img = cv.imread("images/car.jpg")
img_crop = random_center_crop(img, 0.5, 0.8)
display_images([img, img_crop], ("original", "cropped"))

4. Aside from Gaussian noise, name another common type of noise. Write the code to demonstrate how the noise can be included in an image.

In [81]:
# salt-and-pepper noise (only can be applied  to grayscale img )

def add_noise(img):

    # get img dimension
    row, col = img.shape

    # randomly pick pixel between 300 & 5000 to color white (salt noise)
    num_of_pixel = np.random.randint(300, 8000)

    for i in range(num_of_pixel):
        
        # pick random y & x
        y = np.random.randint(0, row-1)
        x = np.random.randint(0, col-1)

        # color the pixel to white
        img[y][x] = 255

    # randomly pick pixel between 300 & 5000 to color black (pepper noise)
    num_of_pixel = np.random.randint(300, 8000)

    for i in range(num_of_pixel):
        
        # pick random y & x
        y = np.random.randint(0, row-1)
        x = np.random.randint(0, col-1)

        # color the pixel to white
        img[y][x] = 0

    return img

In [83]:
img = cv.imread("images/lena.jfif")
img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
img_noise = add_noise(img_gray)

display_images([img, img_noise], ("original", "add noise"))